# MARCH MADNESS PREDICTOR

Authors: Connor Finn, Riley Greene <br>
Date: 1/24/20 <br>
Warren Buffet is still paying 1 billion for a perfect bracket

In [213]:
# our imports for the model
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
import lxml.html as lh

# a few constants
start_year = 2010

Because basketball has changed so much over the past years, we believe that data prior to the year 2010 could be detrimental to our model. For this reason, the next block of code will be used to slim the data down from the 1985 - 2018 seasons to a dataframe including the the 2020 - 2018 NCAA tournaments.

In [214]:
# read ncaa tournament results into dataframe ( data obtained from Kaggle)
ncaa_total = pd.read_csv('NCAATourneyCompactResults.csv')  
# read team ID dataframe (from Kaggle)
team_data = pd.read_csv('teams.csv')  
# get the first index for the desired year
start = min(ncaa_total[ncaa_total.iloc[:,0] == start_year].index.values.astype(int)) 
# shorten our dataframe
ncaa_short = ncaa_total.iloc[start: , :]

The team_id provides names written as 'Penn State'. The data we will be collecting is from Sports-reference.com. The url for penn state's data is 'https://www.sports-reference.com/cbb/schools/penn-state/2020.html' <br>
1) the name needs to be adjusted to no spaces, all lower caps <br>
2) the overall framework is 'https://www.sports-reference.com/cbb/schools/NAME/YEAR.html'<br>

In [238]:
"""
* this function will be used to transform a provided name to one that can be input into a sports-reference url
* this will likely need to be adjusted as new teams are included
* THIS NEEDS A UNIT TEST (i.e. run all our team names through once to see if they pass)
"""
def clean_team_name(name):
    # name: string
    team = name
    team = '!' + team + '!'                   # first and last character
    team = team.replace(' ' , '-')            # no spaces
    team = team.lower()
    team = team.replace('ark-' , 'arkansas-')
    team = team.replace('-st!' , '-state!')
    team = team.replace('!n-' , '!north-')
    team = team.replace('!e-' , '!eastern-')
    team = team.replace('!cs-' , '!cal-state-')
    team = team.replace('!s-' , '!south-')
    team = team.replace('!w-' , '!western-')
    team = team.replace('!nc-' , '!north-carolina-')
    team = team.replace('chr!' , 'christian!')
    team = team.replace('so!' , 'southern!')
    team = team.replace('!fl-' , '!florida-')
    team = team.replace('!uc-' , '!california-')
    team = team.replace('intl!' , 'international!')
    team = team.replace('cal-state-sacramento' , 'sacramento-state')
    team = team.replace('univ!' , 'university!')
    team = team.replace('st-' , 'saint-')
    team = team.replace('miss!' , 'missippi!')
    team = team.replace('-slo' , '')
    team = team.replace('(' , '')
    team = team.replace(')' , '')
    team = team.replace('.' , '')
    team = team.replace( "'" , '')
    team = team.replace("&" , "")
    team = team.replace('--' , '-')
    team = team.replace("university-of-" , "")
    team = team[1:-1]
    
    return team
def test_team_names(team_names):
    # team_names: list of strings 
    errors = 0
    for name in team_names:
        url_test =  "https://www.sports-reference.com/cbb/schools/" + name + "/index.html"
        page_test = requests.get(url_test)   # scrape
        soup_test = BeautifulSoup(page_test.content, "lxml")   # parse
        tables_test = soup_test.findAll('table')    # search for tables 
        if len(tables_test) != 0:
            print("                                 " , name , " Pass")
        else:
            print(name , " Fail")
            errors += 1
    print("finished unit test. There were " , errors , " errors.")
    return errors

team_data is a dataframe which has the columns: 'TeamID', 'TeamName', 'FirstD1Season', 'LastD1Season'.  We want to create a new column which has the team names in accordance to the sports-reference.com framework. 

In [239]:
# get a list of all the team names in our dataframe
team_names = team_data.TeamName.values.tolist()
# clean every name in the team_data dataframe
sr_names = []
for name in team_names:
    sr_names += [clean_team_name(name)]
# run the test: if no error messages show up, then we add the list to dataframe
num_errors = test_team_names(sr_names)
#num_errors = 0
if num_errors ==0:
    team_data["SrNames"] = sr_names

                                  abilene-christian  Pass
                                  air-force  Pass
                                  akron  Pass
                                  alabama  Pass
                                  alabama-am  Pass
                                  alabama-state  Pass
                                  albany-ny  Pass
                                  alcorn-state  Pass
                                  alliant-international  Pass
american-university  Fail
                                  appalachian-state  Pass
                                  arizona  Pass
                                  arizona-state  Pass
                                  arkansas-little-rock  Pass
                                  arkansas-pine-bluff  Pass
                                  arkansas  Pass
                                  arkansas-state  Pass
armstrong-state  Fail
                                  army  Pass
                                  auburn  Pass
aug

                                  mississippi-state  Pass
                                  missouri  Pass
missouri-kc  Fail
                                  missouri-state  Pass
monmouth-nj  Fail
                                  montana  Pass
                                  montana-state  Pass
                                  morehead-state  Pass
                                  morgan-state  Pass
                                  morris-brown  Pass
ms-valley-state  Fail
mt-saint-marys  Fail
mtsu  Fail
                                  murray-state  Pass
north-colorado  Fail
                                  north-dakota-state  Pass
north-illinois  Fail
north-kentucky  Fail
                                  navy  Pass
                                  north-carolina-at  Pass
                                  north-carolina-central  Pass
                                  north-carolina-state  Pass
ne-illinois  Fail
ne-omaha  Fail
                                  nebraska  Pass
 

Goal: compile season long data for the teams who competed in the 2010 - 2018 NCAA tournaments. The idea is that the season long data is the information we will have in the future for creating predictions.<br>
- in the ncaa_short, we have the year the game was played, and the id's of both teams
- in team_data we have the team id, along with the team name (cleaned for sports-reference use).

In [164]:
# get arrays which include the team years and id's 
winner_array = np.vstack((ncaa_short.Season.values , ncaa_short.WTeamID.values ))
loser_array = np.vstack((ncaa_short.Season.values , ncaa_short.LTeamID.values))

# List of years
years = ncaa_short.Season.unique()

# the below loop will create a 2 x n array of all unique teams (year , school_id) which competed in NCAA's
all_teams = np.empty((2,0)) # fill this array
for year in years:
    temp_1 = ncaa_short.WTeamID.loc[(ncaa_short.Season == year)].append(ncaa_short.LTeamID.loc[(ncaa_short.Season == year)]).unique()
    temp_2 = np.full(shape = len(temp_1), fill_value = year , dtype = np.int)
    temp_3 = np.vstack((temp_2 , temp_1)) 
    all_teams = np.hstack((all_teams , temp_3)) # fill array
    

We now want to build a dataframe which has a list of the teams and their season long statistics. <br> 
all_teams is now a 2xn array of unique teams (year, school_id). We will gain access to the team name using: <br>
- team_name = team_data.SrNames.loc[(team_data.TeamID == INSERT TEAM ID HERE)].values.tolist()[0]

In [175]:
# this will collect the season data for every team we selected in the list, for the number of indicated years
seasonStats = pd.DataFrame()   # start with an empty dataframe

for i in range(all_teams.shape[1]):   # this is the years 
	team_name = team_data.SrNames.loc[(team_data.TeamID == all_teams[1 , i])].values.tolist()[0] # get the team Name
	url3 = "https://www.sports-reference.com/cbb/schools/" + str(team_name) + "/" + str(int(all_teams[0 , i])) + ".html" # season data
	pageSe = requests.get(url3)   # scrape
	soupSe = BeautifulSoup(pageSe.content, "lxml")   # parse
	tablesSe = soupSe.findAll('table')    # search for tables 
	print(tablesSe)
	# convert to a dataframe and label the data
	print("Team number " ,  i)   # this shows progress
	dfSe = pd.read_html(str(tablesSe[1]))[0]      # select the table of interest into a pandas dataframe
	dfSe.drop([1 , 2, 3] , inplace=True)	      # only want the team data. (although might consider fouls against too)
	dfSe = dfSe.replace('Team' , powerSixTeams[j])   # want the name of the team 
	numRows = len(dfSe.index) # get the number of rows...... should just be one here
	dfSe['Team_ID'] = powerSixTeamID[j]        # put in the team ID's
	dfSe['Conf_ID'] = powerSixTeamsAndConf[j][1]        # put in the conference ID's  
	dfSe['Conf'] = powerSixConf[powerSixTeamsAndConf[j][1] - 1]       # put in the team's conference
	currYear = [i] * numRows  ## get the correct number 
	dfSe['Date'] = currYear   # only want the year in this column
	seasonStats = seasonStats.append(dfSe , ignore_index=True) # add this to the season stats empty dataframe we started with 
	seasonStats = seasonStats.rename(columns={'Unnamed: 0': 'Team'})  # simply clean up the columns 
    
    

https://www.sports-reference.com/cbb/schools/ark-pine-bluff/2010.html
[]
Team number  0


IndexError: list index out of range

We are interested in using a regression ML model. To do this, we need to have a continuous solution (i.e. not win or lose) So I will assign a scoreDiff

In [ ]:
ncaa_short['ScoreDiff'] = ncaa_short['WScore'] - ncaa_short['LScore']

In [177]:
team_data

,TeamID,TeamName,FirstD1Season,LastD1Season,SrNames
0,1101,Abilene Chr,2014,2019,abilene-chr
1,1102,Air Force,1985,2019,air-force
2,1103,Akron,1985,2019,akron
3,1104,Alabama,1985,2019,alabama
4,1105,Alabama A&M,2000,2019,alabama-am
5,1106,Alabama St,1985,2019,alabama-st
6,1107,Albany NY,2000,2019,albany-ny
7,1108,Alcorn St,1985,2019,alcorn-st
8,1109,Alliant Intl,1985,1991,alliant-intl
9,1110,American Univ,1985,2019,american-univ


In [207]:
v = 'dog'


In [208]:
v

'dog'

In [211]:
v[1:-1]

'o'